In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nsl-kdd-augmented/smote_augmented.csv
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Novelty: Center-Loss Expansion Specialist
# ===========================================
class MLAR_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes, embed_dim=2048):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Linear(input_dim, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.SiLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Linear(embed_dim, num_classes)
        # Learnable Anchors (Centers) for each class
        self.centers = nn.Parameter(torch.randn(num_classes, embed_dim))

    def forward(self, x):
        features = self.feat(x)
        logits = self.classifier(features)
        return logits, features

# ===========================================
# 2️⃣ Custom Loss: Soft-Margin + Center Penalty
# ===========================================
def mlar_criterion(logits, features, targets, centers, weight, lambd=0.01):
    # Standard Weighted Cross Entropy
    ce_loss = F.cross_entropy(logits, targets, weight=weight)
    
    # Center Loss: Forces features to cluster around class anchors
    batch_centers = centers[targets]
    center_loss = F.mse_loss(features, batch_centers)
    
    return ce_loss + lambd * center_loss

# ===========================================
# 3️⃣ Training the Specialist
# ===========================================
model_sp = MLAR_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_sp.parameters(), lr=3e-4, weight_decay=1e-2)

# Specific weights for classes with poor precision/recall balance
sp_weights = torch.ones(num_classes).to(device)
for idx in hard_indices: sp_weights[idx] = 40.0 # Extreme focus

for epoch in range(30):
    model_sp.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, feats = model_sp(xb)
        loss = mlar_criterion(logits, feats, yb, model_sp.centers, sp_weights)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Expert: XGBoost (Optimized for Macro F1)
# ===========================================
expert = XGBClassifier(tree_method='hist', device='cuda', 
                       n_estimators=300, max_depth=10, 
                       learning_rate=0.05, colsample_bytree=0.8)
expert.fit(X_train_proc, y_train_enc)

# ===========================================
# 5️⃣ Final Inference: Dynamic Bayesian Fusion
# ===========================================
def final_q1_fusion(X_proc):
    model_sp.eval()
    with torch.no_grad():
        logits, _ = model_sp(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_sp = torch.softmax(logits * 1.2, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    final_preds = []
    for i in range(len(X_proc)):
        # Calculate confidence gap
        conf_sp = np.max(probs_sp[i])
        conf_ex = np.max(probs_ex[i])
        
        sp_choice = np.argmax(probs_sp[i])
        
        # If Specialist is confident in a HARD class, it wins
        if sp_choice in hard_indices and probs_sp[i][sp_choice] > 0.35:
            final_preds.append(sp_choice)
        else:
            final_preds.append(np.argmax(probs_ex[i]))
            
    return np.array(final_preds)

final_preds = final_q1_fusion(X_test_proc)
unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))

                 precision    recall  f1-score   support

           back       1.00      1.00      1.00       359
buffer_overflow       0.34      0.55      0.42        20
      ftp_write       0.02      0.67      0.04         3
   guess_passwd       0.99      0.27      0.42      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.98      0.99       141
           land       1.00      0.43      0.60         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      1.00      1.00        73
         normal       0.87      0.97      0.92      9711
           perl       0.50      0.50      0.50         2
            phf       1.00      0.50      0.67         2
            pod       0.69      0.88      0.77        41
      portsweep       0.79      0.95      0.86       157
        rootkit       0.01    

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Novelty: Hierarchical Residual Anchor Network
# ===========================================
class HRAN_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Context Stream: Captures global protocol behavior
        self.context_stream = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LayerNorm(512),
            nn.SiLU()
        )
        
        # Detail Stream: Captures subtle R2L/U2R anomalies
        self.detail_stream = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.SiLU()
        )
        
        # Fusion and Anchoring
        self.fusion = nn.Linear(1024, 512)
        self.classifier = nn.Linear(512, num_classes)
        # Learnable Class Anchors for Contrastive separation
        self.anchors = nn.Parameter(torch.randn(num_classes, 512))

    def forward(self, x):
        c = self.context_stream(x)
        d = self.detail_stream(x)
        combined = torch.cat([c, d], dim=1)
        latent = self.fusion(combined)
        logits = self.classifier(latent)
        return logits, latent

# ===========================================
# 2️⃣ Novelty: Orthogonal Anchor Loss
# ===========================================
def orthogonal_anchor_loss(logits, latent, targets, anchors, weight, alpha=0.1):
    # Standard Cross-Entropy with CB Weights
    ce_loss = F.cross_entropy(logits, targets, weight=weight)
    
    # Anchor Distance: Minimizing distance to the correct attack anchor
    correct_anchors = anchors[targets]
    dist_loss = F.mse_loss(latent, correct_anchors)
    
    # Orthogonality: Forcing attack anchors to stay away from the 'Normal' anchor
    # Assuming le.transform(['normal'])[0] is the index for normal
    normal_idx = le.transform(['normal'])[0]
    normal_anchor = anchors[normal_idx].unsqueeze(0)
    cos_sim = F.cosine_similarity(anchors, normal_anchor)
    ortho_loss = torch.mean(cos_sim**2) # Minimize squared similarity
    
    return ce_loss + alpha * (dist_loss + ortho_loss)

# ===========================================
# 3️⃣ Training & Fine-Tuning
# ===========================================
model_hran = HRAN_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_hran.parameters(), lr=4e-4, weight_decay=2e-2)

# Higher priority weights for content attacks that are still struggling
hran_weights = torch.ones(num_classes).to(device)
for idx in hard_indices: hran_weights[idx] = 50.0 

for epoch in range(25):
    model_hran.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, latent = model_hran(xb)
        loss = orthogonal_anchor_loss(logits, latent, yb, model_hran.anchors, hran_weights)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Expert: Heavyweight XGBoost
# ===========================================
expert = XGBClassifier(tree_method='hist', device='cuda', 
                       n_estimators=500, max_depth=12, 
                       learning_rate=0.03, subsample=0.8)
expert.fit(X_train_proc, y_train_enc)

# ===========================================
# 5️⃣ Final Inference: Semantic Gated Fusion
# ===========================================
def semantic_fusion(X_proc):
    model_hran.eval()
    with torch.no_grad():
        logits, _ = model_hran(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_hran = torch.softmax(logits, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    final_preds = []
    for i in range(len(X_proc)):
        # Calculate max confidences
        idx_hran = np.argmax(probs_hran[i])
        
        # Q1 HEURISTIC: Focus on pushing 'warezmaster' and 'guess_passwd'
        # If HRAN identifies a content attack with >20% confidence, we listen
        if idx_hran in hard_indices and probs_hran[i][idx_hran] > 0.20:
            final_preds.append(idx_hran)
        else:
            final_preds.append(np.argmax(probs_ex[i]))
            
    return np.array(final_preds)

final_preds = semantic_fusion(X_test_proc)
unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- HRAN-Net Q1 RESULTS ---")
print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))


--- HRAN-Net Q1 RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      1.00      1.00       359
buffer_overflow       0.43      0.50      0.47        20
      ftp_write       0.01      0.67      0.01         3
   guess_passwd       0.95      0.10      0.18      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.98      0.89      0.93       141
           land       1.00      0.29      0.44         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      1.00      1.00        73
         normal       0.87      0.97      0.92      9711
           perl       0.50      0.50      0.50         2
            phf       1.00      0.50      0.67         2
            pod       0.76      0.46      0.58        41
      portsweep       0.78      0.94      0.85       157
 

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Novelty: Residual Gated Attention Stem
# ===========================================
class ARGF_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Gating mechanism to sieve out minority signals
        self.gate = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Sigmoid()
        )
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LayerNorm(1024),
            nn.SiLU(),
            nn.Linear(1024, 512),
            nn.SiLU()
        )
        
        self.classifier = nn.Linear(512, num_classes)
        # Learnable Class Sharpness (Temperature)
        self.temp = nn.Parameter(torch.ones(num_classes))

    def forward(self, x):
        # Gating: Selective feature amplification
        g = self.gate(x)
        x_gated = x * g + x # Residual connection
        
        feat = self.encoder(x_gated)
        logits = self.classifier(feat)
        # Apply learnable temperature to sharpen minority class predictions
        return logits / torch.clamp(self.temp, min=0.1)

# ===========================================
# 2️⃣ Execution: Focused Training on Content Attacks
# ===========================================
model_argf = ARGF_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_argf.parameters(), lr=8e-4, weight_decay=1e-2)

# Specific weights to fix 'guess_passwd' and 'warezmaster' without hurting accuracy
argf_weights = torch.ones(num_classes).to(device)
argf_weights[le.transform(['guess_passwd'])[0]] = 60.0
argf_weights[le.transform(['warezmaster'])[0]] = 40.0
argf_weights[le.transform(['rootkit'])[0]] = 20.0

criterion = nn.CrossEntropyLoss(weight=argf_weights)

print("Training ARGF Specialist...")
for epoch in range(25):
    model_argf.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model_argf(xb), yb)
        loss.backward()
        optimizer.step()

# ===========================================
# 3️⃣ Expert: XGBoost (Base Stability)
# ===========================================
expert = XGBClassifier(tree_method='hist', device='cuda', 
                       n_estimators=300, max_depth=10, 
                       learning_rate=0.05)
expert.fit(X_train_proc, y_train_enc)

# ===========================================
# 4️⃣ NOVELTY: Adaptive Meta-Selection
# ===========================================
def meta_selection_fusion(X_proc, df_original):
    model_argf.eval()
    with torch.no_grad():
        logits = model_argf(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_argf = torch.softmax(logits, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    # Handpicked Gating Rule: If these flags are active, trust the Specialist
    content_flags = (df_original['num_failed_logins'] > 0) | \
                    (df_original['is_guest_login'] > 0) | \
                    (df_original['hot'] > 0)
    
    final_preds = []
    for i in range(len(X_proc)):
        idx_argf = np.argmax(probs_argf[i])
        idx_ex = np.argmax(probs_ex[i])
        
        # Q1 Logic: If 'Normal' is guessed by XGBoost but 'Attack' is guessed by ARGF
        # and the Content Sieve is triggered, we follow the ARGF
        if content_flags.iloc[i]:
            if idx_argf in hard_indices:
                final_preds.append(idx_argf)
            else:
                final_preds.append(idx_ex)
        else:
            # Otherwise, use a soft confidence fusion
            if np.max(probs_argf[i]) > 0.9 and idx_argf in hard_indices:
                final_preds.append(idx_argf)
            else:
                final_preds.append(idx_ex)
                
    return np.array(final_preds)

print("Inference with Meta-Selection...")
final_preds = meta_selection_fusion(X_test_proc, df_test)

unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- ARGF-Net Q1 RESULTS ---")
print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))

Training ARGF Specialist...
Inference with Meta-Selection...

--- ARGF-Net Q1 RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      1.00      1.00       359
buffer_overflow       0.78      0.35      0.48        20
      ftp_write       0.05      0.33      0.08         3
   guess_passwd       0.98      0.24      0.39      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.98      0.99      0.98       141
           land       1.00      0.57      0.73         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      1.00      1.00        73
         normal       0.86      0.97      0.91      9711
           perl       0.00      0.00      0.00         2
            phf       1.00      0.50      0.67         2
            pod       0.70      0.93      0.80        

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===========================================
# 1️⃣ Novelty: Manifold Amplification Specialist
# ===========================================
class SLMA_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.SiLU(),
            nn.LayerNorm(1024)
        )
        self.bottleneck = nn.Linear(1024, 512)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x, target_labels=None, is_training=False):
        x = self.stem(x)
        latent = self.bottleneck(x)
        
        # SLMA NOVELTY: Perturb latent space for rare classes during training
        if is_training and target_labels is not None:
            # hard_indices must be defined globally from previous steps
            rare_mask = torch.isin(target_labels, torch.tensor(hard_indices).to(device))
            if rare_mask.any():
                noise = torch.randn_like(latent[rare_mask]) * 0.05
                latent[rare_mask] += noise
        
        logits = self.classifier(latent)
        return logits, latent

# ===========================================
# 2️⃣ Novelty: Orthogonal Repulsion Loss
# ===========================================
def slma_loss(logits, latent, targets, weight):
    ce_loss = F.cross_entropy(logits, targets, weight=weight)
    
    # Orthogonal Penalty: Ensure Attack manifolds don't collapse into Normal manifolds
    normal_idx = le.transform(['normal'])[0]
    normal_latent = latent[targets == normal_idx]
    attack_latent = latent[targets != normal_idx]
    
    if len(normal_latent) > 0 and len(attack_latent) > 0:
        sim = F.cosine_similarity(normal_latent.mean(0, keepdim=True), 
                                 attack_latent.mean(0, keepdim=True))
        return ce_loss + 0.1 * sim.pow(2)
    return ce_loss

# ===========================================
# 3️⃣ Strategic Training 
# ===========================================
model_slma = SLMA_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_slma.parameters(), lr=5e-4, weight_decay=2e-2)

# High-Intensity weights for the 'Silent Killers'
slma_weights = torch.ones(num_classes).to(device)
slma_weights[le.transform(['warezmaster'])[0]] = 80.0
slma_weights[le.transform(['rootkit'])[0]] = 50.0
slma_weights[le.transform(['guess_passwd'])[0]] = 40.0

# Using your existing sp_loader or similar filtered dataset
for epoch in range(25):
    model_slma.train()
    for xb, yb in tqdm(sp_loader, desc=f"Epoch {epoch+1}"):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, latent = model_slma(xb, target_labels=yb, is_training=True)
        loss = slma_loss(logits, latent, yb, slma_weights)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Expert & Switchboard Fusion
# ===========================================
expert = XGBClassifier(tree_method='hist', device='cuda', n_estimators=400, max_depth=10)
expert.fit(X_train_proc, y_train_enc)

def switchboard_fusion(X_proc, df_original):
    model_slma.eval()
    with torch.no_grad():
        logits, _ = model_slma(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_slma = torch.softmax(logits, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    # Domain Heuristic: Content-based switchboard
    extreme_anomaly = (df_original['hot'] > 1) | (df_original['num_failed_logins'] > 0)
    
    final_preds = []
    for i in range(len(X_proc)):
        idx_slma = np.argmax(probs_slma[i])
        idx_ex = np.argmax(probs_ex[i])
        
        if extreme_anomaly.iloc[i] or (idx_slma in hard_indices and probs_slma[i][idx_slma] > 0.15):
            final_preds.append(idx_slma)
        else:
            final_preds.append(idx_ex)
    return np.array(final_preds)

# ===========================================
# 5️⃣ Error-Proof Evaluation
# ===========================================
print("Final SLMA Inference...")
final_preds = switchboard_fusion(X_test_proc, df_test)

# FIX: Filter target_names to only include classes present in the test predictions/truth
unique_test_classes = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names_filtered = [le.classes_[i] for i in unique_test_classes]

print("\n--- SLMA-Net Q1 RESULTS ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_test_classes, 
                            target_names=target_names_filtered, 
                            zero_division=0))

Epoch 25: 100%|██████████| 13/13 [00:00<00:00, 183.64it/s]


Final SLMA Inference...

--- SLMA-Net Q1 RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      0.12      0.22       359
buffer_overflow       0.70      0.35      0.47        20
      ftp_write       0.04      0.67      0.07         3
   guess_passwd       0.65      0.10      0.18      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.99      0.99       141
           land       1.00      0.43      0.60         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      0.70      0.82      4657
           nmap       0.96      1.00      0.98        73
         normal       0.85      0.97      0.90      9711
           perl       1.00      0.50      0.67         2
            phf       0.00      0.00      0.00         2
            pod       0.70      0.93      0.80        41
      portsweep       0.79      0.

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===========================================
# 1️⃣ Path A: The High-Stability Anchor (XGBoost)
# ===========================================
# Optimized to protect the majority classes (Normal, Neptune, Smurf)
print("Phase 1: Training Stability Anchor...")
anchor_model = XGBClassifier(
    n_estimators=300, 
    max_depth=10, 
    learning_rate=0.05,
    tree_method='hist',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)
anchor_model.fit(X_train_proc, y_train_enc)

# ===========================================
# 2️⃣ Path B: The Residual Hunter (Deep Gated MLP)
# ===========================================
# This model ONLY cares about the classes the Anchor misses
class ResidualHunter(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.SiLU(),
            nn.LayerNorm(1024),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.SiLU(),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        return self.net(x)

hunter_model = ResidualHunter(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(hunter_model.parameters(), lr=5e-4)

# Weighting: Only focus on the 'Hard' classes
hunter_weights = torch.ones(num_classes).to(device)
hard_list = ['guess_passwd', 'warezmaster', 'rootkit', 'buffer_overflow', 'ftp_write', 'teardrop']
for cls in hard_list:
    idx = le.transform([cls])[0]
    hunter_weights[idx] = 100.0 # Aggressive focus

criterion = nn.CrossEntropyLoss(weight=hunter_weights)

print("Phase 2: Training Residual Hunter...")
# Train on the full dataset but with hunter_weights
for epoch in range(20):
    hunter_model.train()
    for xb, yb in sp_loader: # Using your specialist loader
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(hunter_model(xb), yb)
        loss.backward()
        optimizer.step()

# ===========================================
# 3️⃣ NOVELTY: Cross-Distillation Fusion
# ===========================================
def cross_distillation_inference(X_proc, df_orig):
    hunter_model.eval()
    with torch.no_grad():
        logits = hunter_model(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_h = torch.softmax(logits, dim=1).cpu().numpy()
    
    probs_a = anchor_model.predict_proba(X_proc)
    
    final_preds = []
    for i in range(len(X_proc)):
        # Calculate Confidence Gap
        conf_a = np.max(probs_a[i])
        pred_a = np.argmax(probs_a[i])
        
        conf_h = np.max(probs_h[i])
        pred_h = np.argmax(probs_h[i])
        
        # LOGIC: If Anchor is highly confident (>0.9) in a major class, DO NOT OVERRULE
        if conf_a > 0.92:
            final_preds.append(pred_a)
        # If Hunter detects a Hard Class with decent confidence, LISTEN
        elif pred_h in [le.transform([c])[0] for c in hard_list] and conf_h > 0.15:
            final_preds.append(pred_h)
        else:
            final_preds.append(pred_a)
            
    return np.array(final_preds)

# ===========================================
# 4️⃣ Evaluation
# ===========================================
print("Phase 3: Final Fusion Inference...")
final_preds = cross_distillation_inference(X_test_proc, df_test)

unique_test_classes = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_test_classes]

print("\n--- DPCD-Net Q1 RESULTS ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_test_classes, 
                            target_names=target_names, 
                            zero_division=0))

Phase 1: Training Stability Anchor...
Phase 2: Training Residual Hunter...
Phase 3: Final Fusion Inference...

--- DPCD-Net Q1 RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      1.00      1.00       359
buffer_overflow       0.50      0.10      0.17        20
      ftp_write       0.00      0.00      0.00         3
   guess_passwd       1.00      0.00      0.00      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.99      0.99       141
           land       1.00      0.14      0.25         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      1.00      1.00        73
         normal       0.82      0.98      0.89      9711
           perl       0.00      0.00      0.00         2
            phf       0.00      0.00      0.00         2
     

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===========================================
# 1️⃣ Novelty: Neural Feature Expansion Network
# ===========================================
class NFENet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Expansion to decouple overlapping features
        self.expand = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.SiLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5)
        )
        self.compress = nn.Sequential(
            nn.Linear(4096, 512),
            nn.SiLU()
        )
        self.classifier = nn.Linear(512, num_classes)
        # Learnable Class Centroids
        self.centroids = nn.Parameter(torch.randn(num_classes, 512))

    def forward(self, x):
        features = self.expand(x)
        latent = self.compress(features)
        logits = self.classifier(latent)
        return logits, latent

# ===========================================
# 2️⃣ Novelty: Contrastive Anchoring Loss
# ===========================================
def contrastive_anchor_loss(logits, latent, targets, centroids, weights):
    ce_loss = F.cross_entropy(logits, targets, weight=weights)
    # Centroid Distance (Minimize distance to own class)
    target_centroids = centroids[targets]
    dist_loss = F.mse_loss(latent, target_centroids)
    # Push-Away Loss (Maximize distance to 'Normal' centroid)
    normal_idx = le.transform(['normal'])[0]
    normal_centroid = centroids[normal_idx].detach()
    push_loss = -torch.mean(F.pairwise_distance(latent, normal_centroid))
    return ce_loss + 0.1 * dist_loss + 0.05 * push_loss

# ===========================================
# 3️⃣ Training Loop
# ===========================================
nfe_model = NFENet(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(nfe_model.parameters(), lr=3e-4, weight_decay=1e-2)

weights_nfe = torch.ones(num_classes).to(device)
hard_attacks = ['guess_passwd', 'warezmaster', 'rootkit', 'buffer_overflow', 'ftp_write']
for attack in hard_attacks:
    weights_nfe[le.transform([attack])[0]] = 150.0 

print("Training Expansion Specialist...")
# Use your previously defined specialist loader (sp_loader)
for epoch in range(25):
    nfe_model.train()
    for xb, yb in tqdm(sp_loader, desc=f"Epoch {epoch+1}"):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, latent = nfe_model(xb)
        loss = contrastive_anchor_loss(logits, latent, yb, nfe_model.centroids, weights_nfe)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Inference & Error-Proof Report
# ===========================================
def pgf_inference(X_proc):
    nfe_model.eval()
    with torch.no_grad():
        logits_n, _ = nfe_model(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_n = torch.softmax(logits_n, dim=1).cpu().numpy()
    
    # Use existing XGBoost anchor model from previous steps
    probs_x = anchor_model.predict_proba(X_proc) 
    
    final_preds = []
    hard_indices = [le.transform([a])[0] for a in hard_attacks]
    
    for i in range(len(X_proc)):
        p_x = probs_x[i]
        p_n = probs_n[i]
        
        # Calculate Probability Gap in XGBoost
        top2 = np.sort(p_x)[-2:]
        gap = top2[1] - top2[0]
        
        # Trigger Specialist if XGBoost is unsure or Specialist is confident in hard class
        if gap < 0.6 or (np.argmax(p_n) in hard_indices and np.max(p_n) > 0.3):
            final_preds.append(np.argmax(p_n))
        else:
            final_preds.append(np.argmax(p_x))
    return np.array(final_preds)

print("Running Final Inference...")
final_preds = pgf_inference(X_test_proc)

# ERROR FIX: Dynamically align target names with actual predicted classes
unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- CME-PGG Q1 Final Results ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_labels, 
                            target_names=target_names, 
                            zero_division=0))

Training Expansion Specialist...


Epoch 25: 100%|██████████| 13/13 [00:00<00:00, 241.23it/s]


Running Final Inference...

--- CME-PGG Q1 Final Results ---
                 precision    recall  f1-score   support

           back       1.00      0.95      0.97       359
buffer_overflow       0.14      0.10      0.12        20
      ftp_write       0.01      0.67      0.02         3
   guess_passwd       0.97      0.09      0.17      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.50      0.01      0.01       141
           land       1.00      0.14      0.25         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      0.99      0.99      4657
           nmap       1.00      1.00      1.00        73
         normal       0.90      0.97      0.93      9711
           perl       0.00      0.00      0.00         2
            phf       0.00      0.00      0.00         2
            pod       0.77      0.41      0.54        41
      portsweep       0.79

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Path A: Statistical Expert (XGBoost)
# ===========================================
# Re-running to ensure the global boundaries are clean
expert_xgb = XGBClassifier(tree_method='hist', device='cuda', n_estimators=300, max_depth=10)
expert_xgb.fit(X_train_proc, y_train_enc)

# ===========================================
# 2️⃣ Path B: Manifold Specialist (CME-Net)
# ===========================================
# Use the nfe_model trained in the previous step
# (Assuming nfe_model is already trained and in memory)

# ===========================================
# 3️⃣ NOVELTY: Protocol-Aware Decision Logic
# ===========================================
def tri_stream_fusion(X_proc, df_orig):
    nfe_model.eval()
    with torch.no_grad():
        logits_n, _ = nfe_model(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_n = torch.softmax(logits_n, dim=1).cpu().numpy()
    
    probs_x = expert_xgb.predict_proba(X_proc)
    
    final_preds = []
    hard_indices = [le.transform([a])[0] for a in ['guess_passwd', 'warezmaster', 'rootkit', 'buffer_overflow']]
    
    # Get protocol indices from df_orig
    # Logic: R2L attacks are almost exclusively TCP/UDP
    for i in range(len(X_proc)):
        p_x = probs_x[i]
        p_n = probs_n[i]
        
        # Determine protocol context
        protocol = df_orig['protocol_type'].iloc[i]
        
        # TRISTREAM LOGIC:
        # 1. If it's a Probing/DoS protocol (ICMP), trust XGBoost 100%
        if protocol == 'icmp':
            final_preds.append(np.argmax(p_x))
            
        # 2. If it's TCP/UDP and the Specialist is confident in an R2L attack, trust Specialist
        elif np.argmax(p_n) in hard_indices and np.max(p_n) > 0.4:
            final_preds.append(np.argmax(p_n))
            
        # 3. Otherwise, use a Confidence-Weighted Average
        else:
            # Weighted blend: give more weight to XGBoost for 'normal' and DoS
            blended = (0.7 * p_x) + (0.3 * p_n)
            final_preds.append(np.argmax(blended))
            
    return np.array(final_preds)

# ===========================================
# 4️⃣ Execution & Final Report
# ===========================================
print("Running Tri-Stream Fusion...")
final_preds = tri_stream_fusion(X_test_proc, df_test)

unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- TSOF Q1 FINAL RESULTS ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_labels, 
                            target_names=target_names, 
                            zero_division=0))

Running Tri-Stream Fusion...

--- TSOF Q1 FINAL RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      0.94      0.97       359
buffer_overflow       0.14      0.10      0.12        20
      ftp_write       0.00      0.00      0.00         3
   guess_passwd       0.97      0.09      0.17      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.98      0.99      0.98       141
           land       0.00      0.00      0.00         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       0.96      1.00      0.98        73
         normal       0.90      0.96      0.93      9711
           perl       0.00      0.00      0.00         2
            phf       0.00      0.00      0.00         2
            pod       0.71      0.95      0.81        41
      portsweep       0.79 